# Naive Bayes Algorithm

The Naive Bayes classification algorithm is based off of **Bayes’ Theorem**. It is a **supervised** classification algorithm which can be used for **both categorical and numerical feature data**. 

**Bayes’ Theorem** uses **conditional probabilities** to “predict” the outcome of later probabilities. The Bayesian concept of a **priori probability** is important: it is used as a ground truth from which we can use to obtain the outcome of other probabilities. Let’s take a look at Bayes’ theorem:

![title](http://uc-r.github.io/public/images/analytics/naive_bayes/naive_bayes_icon.png)

P(c) indicates the a priori probability of the given class, while P(x) represents the probability of a given predictor. Priori probability is always known a priori from the training examples,posteriori probability is the probability of the class (or model) which we calculate after seeing the data (or evidence or events). Likelihood answers the question that “What is the probability of the data (or evidence or events) that it has been generated from the model (or class or outcome) ?” Please refer to this [blog](https://appliedmachinelearning.blog/2017/05/23/understanding-naive-bayes-classifier-from-scratch-python-code/) for detailed explanation with example.

**Why called Naive??**

The model is very naively believing that the effect of the occurrence of any of the events is completely independent of the occurrence of other events. This is obviously not the case with most of the real world problems and the predictor features are known to influence and be influenced by other features. Still, naive Bayes is known to provide results at par and sometimes even better than highly complex and computationally expensive classification models.

## Implementation of Naive Baye's from scratch for categorical features

As we can see from the formula, we need to calculate class prior probabilities **P(c)**, Likelihood **P(x/c)** and predictor prior probabilities **P(x)** to implement the algorithm and calculate the posterior probability for any given test data. Note that we **donot need** to calculate predictor prior probability **P(x)** as it will remain **same for all posterior probability calculation** for all possible output classes. Since we will be using class with maximum posterior probability as the prediction class, P(X) just **acts as a scaling factor** and can be ignored.

**P(c)** is calculated as:
![title](https://cdn-images-1.medium.com/max/800/1*-PkUQ4n42T1YLaK-jXI9VQ.png)

i.e divide occurrence of given class by sum of all class occurrence.

Likelihood **P(x/c)** is calculated as: 

![title](https://cdn-images-1.medium.com/max/800/1*PB2b5dB2rHffvx_g-wcSng.png)

i.e for a given feature level wi  we count how many of wi belong in class c. We then divide this by ALL the w that belong to c. This gives us a probability for a wi given c. We can make a minor adjustment to this formula to smooth the data (this accounts for the possibility that some feature level wi could have a zero-count for a class, which would make eliminate our work done so far). Please refer to the blog above for better understanding.

Once we have above to information, given a test data point, we can calculate the posterior probability **P(c/x)** and select the highest probability class.

### 1.  Creating Synthetic data

In [1]:
from dataclasses import dataclass
from collections import defaultdict
import operator
from functools import reduce
from operator import itemgetter
from collections import Counter
import numpy as np
import re

In [2]:
@dataclass
class LabeledTextData:
    id_num: int
    label: str
    tokens: list

train = [LabeledTextData(42, 'cat',  "🐈 🐯 🐱 🐩 🐱".split()),
         LabeledTextData(43, 'dog',  "🐶 🐶 🐈 🐶 🐩 🐈 🐶 🐶".split()),
         LabeledTextData(45, 'cat',  "🐈 🐈 🐯 🐶 🐈".split()),
         LabeledTextData(45, 'cat',  "🐈 🐈 🐈".split()),
         LabeledTextData(48, 'dog',  "🐶 🐶 🐯 🐈 🐩 🐱 🐩 🐶 🐩 🐶 ".split()),
        ]

Data class is a way for creating a new data type with custom attributes. Learn more about dataclasses [here](https://realpython.com/python-data-classes/)

### 2. Calculating class priors

In [3]:
def doc_priors(labels):
    doc_priors = defaultdict(float)

    for label in labels:
        doc_priors[label] = sum(1 for d in labels if d == label) / len(labels)

    return doc_priors

In [4]:
labels = [d.label for d in train]
prior = doc_priors(labels)
prior 

defaultdict(float, {'cat': 0.6, 'dog': 0.4})

### 3. Calculating Likelihood

In this step we do following things. We first create a dictionary of various unique features. In our case these are the shapes. If we had a word document, it would have been unique words and the collection of unique words would have been our vocabulary. Once we have created a vocabulary, we calculate the likelihood for each output class.

In [5]:
def likelihood(train, labels):
    
    # A default dict of default dicts; inner default dict is probability
    cond_prob = defaultdict(lambda: defaultdict(float))

    for label in set(labels):
    
        label_tokens = []
        for i, doc in enumerate(train):
             # For a given label, get a list of all the tokens for all the docs 
            if labels[i] == label:
                label_tokens.extend(doc)

        for token in set(label_tokens):
            # Find conditional probability: token count / total count
            cond_prob[label][token] = label_tokens.count(token) / len(label_tokens) 

    return cond_prob

In [6]:
labels
train_x =[x.tokens for x in train]
cond_prob = likelihood(train_x, labels)
cond_prob

defaultdict(<function __main__.likelihood.<locals>.<lambda>()>,
            {'cat': defaultdict(float,
                         {'🐈': 0.5384615384615384,
                          '🐩': 0.07692307692307693,
                          '🐯': 0.15384615384615385,
                          '🐶': 0.07692307692307693,
                          '🐱': 0.15384615384615385}),
             'dog': defaultdict(float,
                         {'🐈': 0.16666666666666666,
                          '🐩': 0.2222222222222222,
                          '🐯': 0.05555555555555555,
                          '🐶': 0.5,
                          '🐱': 0.05555555555555555})})

In [7]:
cond_prob.keys()

dict_keys(['cat', 'dog'])

### 4. Calculating Posterior Probability 

In [8]:
def product(iterable):
    return reduce(operator.mul, iterable, 1)

def post_prob(test, cond_prob, doc_priors):
    prob_predicted = defaultdict(float)
    for label in cond_prob.keys():
        # For each label, calculate the conditional probability based on the prior and the tokens that appear
        prob_predicted[label] = doc_priors[label] * product(cond_prob[label][t] for t in test)
    
    return prob_predicted

In [9]:
test = LabeledTextData(id_num=90, label=None, tokens="🐱".split())
# test = LabeledTextData(id_num=91, label=None, tokens="🐶 🐶".split()) 
# test = LabeledTextData(id_num=92, label=None, tokens="🐶 🐱".split())
# test = LabeledTextData(id_num=93, label=None, tokens="🐈 🐈 🐶 🐶 🐩 🐱 🐱".split())
# test = LabeledTextData(id_num=94, label=None, tokens="🐬 ".split()) # Out of sample prediction
test_x = test.tokens
prob_predicted = post_prob(test_x, cond_prob, prior)
prob_predicted

defaultdict(float, {'cat': 0.09230769230769231, 'dog': 0.022222222222222223})

### 5. Making predictions

In [10]:
label, prob = max(prob_predicted.items(),
                  key=itemgetter(1))
print("The predicted class is:", label)

The predicted class is: cat


## Putting everything together in a class
We will make a class so that the it becomes easier to apply the algorithm and make predictions. We will also make slight changes by using **log soft probability calculation** so the **model performs much better** even for large number of features and we **donot have the issue of multiplying with 0 if a feature is missing.**

In [11]:
class NaiveBayes_MN:
    def __init__(self):
        self.voccab = []
        self.labelset = None
        self.n_items = None
        self.priors = defaultdict(float)
        self.prob_predicted = defaultdict(float)
        self.c_word_dict = defaultdict(lambda: defaultdict(float))
        self.class_count = defaultdict(float)
    
    def fit(self, x_train, labels):
        self.x_train = np.array(x_train)
        self.labels = np.array(labels)
        self.n_items = self.labels.size
        self.labelset = set(self.labels)
        
        word_dict = defaultdict(list)
        for i,doc in enumerate(x_train):
            self.voccab.extend(doc)
            word_dict[labels[i]].extend(doc)
        
        for i in self.labelset:
            self.c_word_dict[i] = Counter(word_dict[i])
            self.class_count[i] = np.sum(list(self.c_word_dict[i].values())) 
        
        self.voccab = set(self.voccab)
        self.voccab_len = len(self.voccab)
        self.doc_priors()

    
    def doc_priors(self):
        for label in self.labelset:
            self.priors[label] = np.sum(1 for d in self.labels if d == label)*1.0 / self.n_items
    
    def likelihood(self, word, label):
        return np.log((self.c_word_dict[label][word]+1.0)/(self.class_count[label]+self.voccab_len + 1.0))

    def post_prob(self, test):
        for label in self.labelset:
            self.prob_predicted[label] = np.log(self.priors[label]) 
            for t in test:
                self.prob_predicted[label] += self.likelihood(t, label)
        return self.prob_predicted

    def predict(self, test):
        self.test_labels = []
        for i in test:
            prob_predicted = self.post_prob(i)
            label, prob = max(prob_predicted.items(),
                      key=itemgetter(1))
            self.test_labels.append(label)
        return self.test_labels
        

### Predicting on previous test points

In [12]:
clf = NaiveBayes_MN()
clf.fit(train_x, labels)

In [13]:
clf.class_count

defaultdict(float, {'cat': 13, 'dog': 18})

In [14]:
test_x

['🐱']

In [15]:
clf.predict([test_x])

['cat']

### Predicting on Real data set

We will use sklearn's 20newsgroups data set and consider only 4 categories to test our implementation

In [16]:
from sklearn.datasets import fetch_20newsgroups
categories=['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med'] 
newsgroups_train=fetch_20newsgroups(subset='train',categories=categories)

train_data=newsgroups_train.data #getting all trainign examples
train_labels=newsgroups_train.target #getting training labels



### Function to tokenize the data and get it in the right format

In [17]:
def preprocess_string(str_arg):
    cleaned_str=re.sub('[^a-z\s]+',' ',str_arg,flags=re.IGNORECASE) #every char except alphabets is replaced
    cleaned_str=re.sub('(\s+)',' ',cleaned_str) #multiple spaces are replaced by single space
    cleaned_str=cleaned_str.lower() #converting the cleaned string to lower case
    
    return cleaned_str # returning the preprocessed string 


In [18]:
train_data = [preprocess_string(x).split() for x in train_data]

#### Applying NaiveBayes

In [19]:
clf = NaiveBayes_MN()
clf.fit(train_data, train_labels)

#### Test set

In [20]:
newsgroups_test=fetch_20newsgroups(subset='test',categories=categories) #loading test data
test_data=newsgroups_test.data #get test set examples
test_labels=newsgroups_test.target #get test set labels

In [21]:
test_data = [preprocess_string(x).split() for x in test_data]

In [22]:
predicted = clf.predict(test_data)

In [23]:
print("Predicted accuracy is %.3f" %(np.sum(test_labels == predicted)/len(predicted)))

Predicted accuracy is 0.939


**We have build a good enough model**

The naive Bayes (NB) classifier is a probabilistic classifier that assumes complete independence between the predictor features. They perform quite well even though the working behind them is simplistic. There exist other methods as well to determine the likelihoods of the features. Thus the NB classifiers are actually a family of classifiers rather being just a single classifier. Below are the classifiers available in any machine learning library, such as sklearn for Python.

**Classification based on feature type**

**Gaussian NB** – The likelihood of the features is assumed to be Gaussian distribution. We will implement **Gaussian NB in a separate notebook**. 

**Multinomial NB** – This works for multinomially distributed data like word count vector where features in the vectors are probability of the event appearing in the sample. The algorithm remains the same. We just pre-calculate the word counts per document and feed it and rest of the steps are same. Interestingly tf-idf values also work in even though they are continuous numbers and the base algorithm does not change.

**Bernoulli NB** – This is used for multivariate Bernoulli distributions; i.e., there may be multiple features but each one is assumed to be a binary-valued (boolean) variable. The only change is in the way conditional probability is calculated and can be easily implemented by just changing the likelihood function.

### Pros and Cons of Naive Bayes

**Pros:**

1. It is easy and fast to predict class of test data set. It also perform well in multi class prediction
2. When assumption of independence holds, a Naive Bayes classifier performs better compare to other models like logistic regression and you need less training data.
3. It perform well in case of categorical input variables compared to numerical variable(s). For numerical variable, normal distribution is assumed (bell curve, which is a strong assumption).

**Cons:**

1. If categorical variable has a category (in test data set), which was not observed in training data set, then model will assign a 0 (zero) probability and will be unable to make a prediction. This is often known as “Zero Frequency”. To solve this, we can use the smoothing technique. One of the simplest smoothing techniques is called Laplace estimation which we used above.
2. On the other side naive Bayes is also known as a bad estimator, so the probability outputs from predict_proba are not to be taken too seriously.
3. Another limitation of Naive Bayes is the assumption of independent predictors. In real life, it is almost impossible that we get a set of predictors which are completely independent.

### Tips to improve the power of Naive Bayes Model


1. If continuous features do not have normal distribution, we should use transformation or different methods to convert it in normal distribution.
2. Remove correlated features, as the highly correlated features are voted twice in the model and it can lead to over inflating importance.
3. Naive Bayes classifiers has limited options for parameter tuning like alpha=1 for smoothing, fit_prior=[True|False] to learn class prior probabilities or not and some other options (look at detail here). I would recommend to focus on your  pre-processing of data and the feature selection.